In [1]:
import numpy as np

class DecisionStump:
    """
    약한 학습기로 사용할 결정 스텀프
    """
    def __init__(self):
        self.feature_idx = None
        self.threshold = None
        self.polarity = 1  # 1: x <= threshold, -1: x > threshold
        self.alpha = None

    def fit(self, X, y, weights):
        """
        결정 스텀프 학습
        :param X: 입력 데이터 (N, d)
        :param y: 타겟 값 (N,)
        :param weights: 샘플 가중치 (N,)
        """
        N, d = X.shape
        min_error = float('inf')

        # 모든 특성과 임계값에 대해 테스트
        for feature in range(d):
            X_feature = X[:, feature]
            thresholds = np.unique(X_feature)

            for threshold in thresholds:
                for polarity in [1, -1]:
                    error = 0
                    # 예측
                    for i in range(N):
                        pred = 1 if polarity * X_feature[i] <= polarity * threshold else -1
                        if pred != y[i]:
                            error += weights[i]

                    # 최소 오차 갱신
                    if error < min_error:
                        min_error = error
                        self.feature_idx = feature
                        self.threshold = threshold
                        self.polarity = polarity

        return min_error

    def predict(self, X):
        """
        결정 스텀프 예측
        :param X: 입력 데이터 (N, d)
        :return: 예측값 (N,)
        """
        X_feature = X[:, self.feature_idx]
        predictions = np.ones(len(X))
        if self.polarity == 1:
            predictions[X_feature <= self.threshold] = -1
        else:
            predictions[X_feature > self.threshold] = -1
        return predictions

class AdaBoost:
    def __init__(self, n_classifiers=5):
        """
        AdaBoost 초기화
        :param n_classifiers: 약한 학습기 개수 (K)
        """
        self.n_classifiers = n_classifiers
        self.classifiers = []
        self.alphas = []

    def fit(self, X, y):
        """
        AdaBoost 학습
        :param X: 훈련 데이터 (N, d)
        :param y: 타겟 값 (N,) {-1, 1}
        """
        N = len(X)
        # 샘플 가중치 초기화 (w_i = 1/N)
        weights = np.ones(N) / N

        for k in range(self.n_classifiers):
            # 약한 학습기 학습
            clf = DecisionStump()
            error = clf.fit(X, y, weights)

            # 오차 계산
            epsilon = error
            if epsilon > 0.5:
                print(f"오차가 0.5보다 큼 (epsilon={epsilon}), 학습 중단")
                break

            # alpha_k 계산
            alpha = 0.5 * np.log((1 - epsilon) / (epsilon + 1e-10))
            self.alphas.append(alpha)
            clf.alpha = alpha

            # 예측
            predictions = clf.predict(X)

            # 가중치 업데이트
            for i in range(N):
                if predictions[i] != y[i]:  # 잘못 분류된 경우
                    weights[i] *= np.exp(alpha)
                else:  # 맞춘 경우
                    weights[i] *= np.exp(-alpha)

            # 가중치 정규화
            weights /= np.sum(weights)

            # 분류기 추가
            self.classifiers.append(clf)
            print(f"분류기 {k+1}: alpha={alpha:.4f}, epsilon={epsilon:.4f}")

    def predict(self, X):
        """
        AdaBoost 예측
        :param X: 입력 데이터 (N, d)
        :return: 예측값 (N,)
        """
        clf_preds = np.zeros(len(X))
        for alpha, clf in zip(self.alphas, self.classifiers):
            clf_preds += alpha * clf.predict(X)
        return np.sign(clf_preds)

# 테스트
if __name__ == "__main__":
    # 간단한 이진 분류 데이터 생성
    np.random.seed(0)
    X = np.array([
        [1, 2], [2, 3], [3, 1], [4, 3],  # 클래스 -1
        [5, 5], [6, 4], [7, 6], [8, 5]   # 클래스 1
    ])
    y = np.array([-1, -1, -1, -1, 1, 1, 1, 1])

    # AdaBoost 학습
    adaboost = AdaBoost(n_classifiers=5)
    adaboost.fit(X, y)

    # 예측
    predictions = adaboost.predict(X)
    print("\n예측 결과:")
    for i in range(len(X)):
        print(f"입력: {X[i]}, 실제: {y[i]}, 예측: {predictions[i]}")

    # 정확도 계산
    accuracy = np.mean(predictions == y)
    print(f"\n정확도: {accuracy * 100:.2f}%")

분류기 1: alpha=11.5129, epsilon=0.0000
분류기 2: alpha=11.5129, epsilon=0.0000
분류기 3: alpha=11.5129, epsilon=0.0000
분류기 4: alpha=11.5129, epsilon=0.0000
분류기 5: alpha=11.5129, epsilon=0.0000

예측 결과:
입력: [1 2], 실제: -1, 예측: 1.0
입력: [2 3], 실제: -1, 예측: 1.0
입력: [3 1], 실제: -1, 예측: 1.0
입력: [4 3], 실제: -1, 예측: 1.0
입력: [5 5], 실제: 1, 예측: 1.0
입력: [6 4], 실제: 1, 예측: -1.0
입력: [7 6], 실제: 1, 예측: -1.0
입력: [8 5], 실제: 1, 예측: -1.0

정확도: 12.50%
